In [74]:
!pip install dash jupyter-dash
!pip install dash-cytoscape



     ---------------------------------------- 0.0/4.0 MB ? eta -:--:--
     ----- ---------------------------------- 0.5/4.0 MB 8.5 MB/s eta 0:00:01
     ------------------ --------------------- 1.8/4.0 MB 6.3 MB/s eta 0:00:01
     --------------------- ------------------ 2.1/4.0 MB 5.1 MB/s eta 0:00:01
     -------------------------- ------------- 2.6/4.0 MB 4.1 MB/s eta 0:00:01
     ------------------------------- -------- 3.1/4.0 MB 3.4 MB/s eta 0:00:01
     ---------------------------------- ----- 3.4/4.0 MB 3.2 MB/s eta 0:00:01
     ---------------------------------------- 4.0/4.0 MB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for dash-cytoscape: filename=dash_cytoscape-1.0.2-py3-none-any.whl size=4010751 sha256=6b3ccd21df9f47aa4889d20dc0bcbbd54557744b09fa304beeab5b77587769d9
  Stored in directory: c:\users\nael_gaizka\appdata\local\pip\cache\wheels\0c\db\f6\9dcb225e9adf45dfef71354276955

In [56]:
import json
import dash
from dash import dcc, html
import plotly.graph_objs as go
from dash import Dash
import os

# Charger le fichier Jupyter Notebook depuis un chemin local
notebook_path = 'Copie_de_projet_tpe2_1.ipynb'
try:
    with open(notebook_path, 'r', encoding='utf-8') as f:
        notebook_data = json.load(f)
except FileNotFoundError:
    print(f"Erreur : Le fichier '{notebook_path}' est introuvable. Veuillez vérifier le chemin du fichier.")
    notebook_data = {'cells': []}
except json.JSONDecodeError as e:
    print(f"Erreur : Impossible de lire le fichier JSON. Veuillez vérifier le contenu du fichier. Détails : {e}")
    notebook_data = {'cells': []}

# Extraire les cellules du notebook
cells = notebook_data.get('cells', [])

# Extraire le code et les sorties
code_cells = [cell for cell in cells if cell['cell_type'] == 'code']
code_outputs = [cell.get('outputs', []) for cell in code_cells]

# Créer une application Dash
app = Dash(__name__)

app.layout = html.Div([
    html.H1('Visualisation du contenu du Notebook'),
    dcc.Tabs([
        dcc.Tab(label='Code', children=[
            html.Div([html.Pre(cell['source']) for cell in code_cells])
        ]),
        dcc.Tab(label='Sorties', children=[
            html.Div([html.Pre(output) for outputs in code_outputs for output in outputs])
        ]),
    ])
])

if __name__ == '__main__':
    app.run_server(port=8050, debug=True)


In [58]:
import json
import dash
from dash import dcc, html
import plotly.graph_objs as go
from dash import Dash
import os

# Charger le fichier Jupyter Notebook depuis un chemin local
notebook_path = 'Copie_de_projet_tpe2_1.ipynb'
try:
    with open(notebook_path, 'r', encoding='utf-8') as f:
        notebook_data = json.load(f)
except FileNotFoundError:
    print(f"Erreur : Le fichier '{notebook_path}' est introuvable. Veuillez vérifier le chemin du fichier.")
    notebook_data = {'cells': []}
except json.JSONDecodeError as e:
    print(f"Erreur : Impossible de lire le fichier JSON. Veuillez vérifier le contenu du fichier. Détails : {e}")
    notebook_data = {'cells': []}

# Extraire les cellules du notebook
cells = notebook_data.get('cells', [])

# Extraire le code, les sorties, les cellules de texte et les images
code_cells = [cell for cell in cells if cell['cell_type'] == 'code']
code_outputs = [cell.get('outputs', []) for cell in code_cells]
markdown_cells = [cell for cell in cells if cell['cell_type'] == 'markdown']
image_outputs = []

for outputs in code_outputs:
    for output in outputs:
        if 'data' in output and 'image/png' in output['data']:
            image_outputs.append(output['data']['image/png'])

# Créer une application Dash
app = Dash(__name__)

app.layout = html.Div([
    html.H1('Visualisation du contenu du Notebook'),
    dcc.Tabs([
        dcc.Tab(label='Code', children=[
            html.Div([html.Pre(cell['source']) for cell in code_cells])
        ]),
        dcc.Tab(label='Sorties', children=[
            html.Div([
                html.Pre(output.get('text', '')) if 'text' in output else html.Div() 
                for outputs in code_outputs for output in outputs
            ])
        ]),
        dcc.Tab(label='Markdown', children=[
            html.Div([html.Div(cell['source']) for cell in markdown_cells])
        ]),
        dcc.Tab(label='Images', children=[
            html.Div([
                html.Img(src='data:image/png;base64,{}'.format(img)) for img in image_outputs
            ])
        ]),
    ])
])

if __name__ == '__main__':
    app.run_server(port=8050, debug=True)


In [82]:
import dash
from dash import dcc, html
import dash_cytoscape as cyto
from dash import Dash
import networkx as nx
from rdflib import Namespace, Graph, URIRef
from rdflib.namespace import RDF
import re
from community import community_louvain

# Charger le fichier RDF
akg_file = "teaching_akg.ttl"
g = Graph()
g.parse(akg_file)
G = nx.Graph()

# Définir les namespaces
akg_namespace = Namespace("http://sonfack.com/2023/12/tao/")
cao_namespace = Namespace("http://sonfack.com/2023/12/cao/")

# Fonction pour extraire et nettoyer les noms des URI
def extract_name(uri):
    """Extrait le nom propre d'un URI en supprimant les namespaces et les identifiants."""
    name = re.sub(r'http://sonfack\.com/2023/12/(tao|cao)/', '', uri)  # Supprimer les namespaces
    name = re.sub(r'-\w+$', '', name)  # Supprimer les identifiants après un tiret
    return name

# Fonction pour lire toutes les activités
def read_all_activities(akg: Graph) -> list:
    """Retourne toutes les activités dans un graphe de connaissances."""
    return [activity for activity in akg.subjects(predicate=RDF.type, object=cao_namespace.Activity)]

# Fonction pour obtenir les détails d'une activité spécifique
def get_activity_details(activity_uri: URIRef, akg: Graph) -> dict:
    """Retourne les détails d'une activité sous forme de dictionnaire."""
    details = {}
    for pred, obj in akg.predicate_objects(subject=activity_uri):
        # Ignorer le type 'Activity' dans l'affichage
        if str(obj) == str(cao_namespace.Activity):
            continue
        pred_label = extract_name(str(pred))
        obj_label = extract_name(str(obj))
        details.setdefault(pred_label, []).append(obj_label)
    return details

# Fonction pour ajouter les activités et leurs relations au graphe NetworkX
def add_activity_to_graph(G, activity_uri, akg):
    """Ajoute une activité et ses relations au graphe NetworkX."""
    details = get_activity_details(activity_uri, akg)
    activity_name = extract_name(str(activity_uri))
    
    for pred, objs in details.items():
        for obj in objs:
            obj_name = extract_name(str(obj))
            G.add_node(activity_name, type="activité", color="orange")  # Couleur orange pour les activités
            G.add_node(obj_name, type="ressource", color="blue")        # Couleur bleue pour les ressources
            G.add_edge(activity_name, obj_name, label=pred)

# Ajouter toutes les activités au graphe NetworkX
all_activities = read_all_activities(g)
for activity in all_activities:
    add_activity_to_graph(G, activity, g)

# Appliquer l'algorithme de Louvain pour détecter les communautés
def apply_louvain_clustering(G):
    """Applique l'algorithme de Louvain pour détecter les communautés dans le graphe."""
    partition = community_louvain.best_partition(G)
    return partition

partition = apply_louvain_clustering(G)

# Convertir le graphe NetworkX en un format compatible avec Cytoscape
cyto_elements = [
    {'data': {'id': str(node), 'label': f'{extract_name(node)}', 'group': partition.get(node, -1)}} for node in G.nodes
] + [
    {'data': {'source': str(edge[0]), 'target': str(edge[1]), 'label': G.edges[edge].get('label', '')}} for edge in G.edges
]

# Créer une application Dash
app = Dash(__name__)

app.layout = html.Div([
    html.H1('Visualisation des Communautés d\'Activités'),
    dcc.Tabs([
        dcc.Tab(label='Communautés', children=[
            html.Div([
                html.H3('Options de visualisation des communautés :'),
                dcc.RadioItems(
                    id='choix-visualisation',
                    options=[
                        {'label': 'Visualiser toutes les communautés séparément', 'value': 'toutes'},
                        {'label': 'Visualiser une seule communauté', 'value': 'une'},
                        {'label': 'Sélectionner 2 activités ou plus', 'value': 'plusieurs'},
                        {'label': 'Détecter les communautés en fonction des ressources', 'value': 'ressources'}
                    ],
                    value='toutes'
                ),
                html.Div(id='contenu-visualisation')
            ])
        ]),
        dcc.Tab(label='Graphe des Communautés', children=[
            html.Div([
                html.H3('Visualisation des communautés sous forme de graphe interactif'),
                cyto.Cytoscape(
                    id='cytoscape-graphe',
                    elements=cyto_elements,
                    style={'width': '100%', 'height': '500px'},
                    layout={'name': 'cose'},
                    stylesheet=[
                        {
                            'selector': 'node',
                            'style': {
                                'label': 'data(label)',
                                'background-color': 'mapData(group, 0, 10, blue, orange)'
                            }
                        },
                        {
                            'selector': 'edge',
                            'style': {
                                'curve-style': 'bezier',
                                'target-arrow-shape': 'triangle',
                                'label': 'data(label)'
                            }
                        }
                    ]
                )
            ])
        ])
    ])
])

@app.callback(
    dash.dependencies.Output('contenu-visualisation', 'children'),
    [dash.dependencies.Input('choix-visualisation', 'value')]
)
def mettre_a_jour_visualisation(choix):
    if choix == 'toutes':
        communautes = {}
        for node, comm in partition.items():
            communautes.setdefault(comm, []).append(node)
        return html.Div([
            html.Div([
                html.H4(f'Communauté {comm + 1}'),
                html.Ul([html.Li(f'Nœud: {extract_name(n)}') for n in nodes])
            ]) for comm, nodes in communautes.items()
        ])
    elif choix == 'une':
        return html.Div([
            dcc.Input(id='input-activite', type='text', placeholder="Entrez le nom de l'activité"),
            html.Button('Visualiser', id='bouton-visualiser'),
            html.Div(id='resultat-une-communaute')
        ])
    elif choix == 'plusieurs':
        return html.Div([
            dcc.Dropdown(
                id='dropdown-activites',
                options=[{'label': extract_name(node), 'value': node} for node in G.nodes],
                multi=True
            ),
            html.Button('Visualiser', id='bouton-visualiser-plusieurs'),
            html.Div(id='resultat-plusieurs-communaute')
        ])
    elif choix == 'ressources':
        return html.Div([
            html.H4('Détection des communautés en fonction des ressources'),
            dcc.Dropdown(
                id='dropdown-ressources',
                options=[
                    {'label': 'hasObjective', 'value': 'hasObjective'},
                    {'label': 'activityDescription', 'value': 'activityDescription'},
                    {'label': 'hasSubject', 'value': 'hasSubject'},
                    {'label': 'hasName', 'value': 'hasName'},
                    {'label': 'hasDuration', 'value': 'hasDuration'},
                    {'label': 'hasBegeinTime', 'value': 'hasBegeinTime'},
                    {'label': 'isComposedOf', 'value': 'isComposedOf'},
                    {'label': 'hasLocation', 'value': 'hasLocation'},
                    {'label': 'schema#', 'value': 'schema#'},
                    {'label': 'hasBeneficialTo', 'value': 'hasBeneficialTo'},
                    {'label': 'isCarriedOutWith', 'value': 'isCarriedOutWith'}
                ],
                multi=True
            ),
            html.Button('Détecter', id='bouton-detecter'),
            html.Div(id='resultat-ressources')
        ])
    else:
        return html.Div('Veuillez sélectionner une option.')

if __name__ == '__main__':
    app.run_server(port=8050, debug=True)
